# A hand-made chi2 fit.

Let's generate a few distributions and calculate the mean, mode, median for each.
Start by importing the relevant libraries

In [ ]:
from math import *
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

Let's generate a distribution we want to fit. We take an exponential

In [ ]:
def generateExpo(tau=1, Nevents=10000, randomSeed=-1):
    if randomSeed >=0: np.random.seed(randomSeed)
    expoSample = np.random.exponential(scale = tau, size=Nevents)
    return expoSample

In [ ]:
Nevents=1000
Nbins=30
histoRange=(0,4)
expoSample = generateExpo(1.5, Nevents)
timeHisto=plt.hist(expoSample, Nbins, histoRange) # make histogram
plt.show()

Define the theoretical distribution

In [ ]:
def expo(t, tau):
    return np.exp(-t/tau)/tau
def expoIntegral(t, tau, integRange=(0.0, -999.9)):
    lower=np.exp(-integRange[0]/tau)
    upper=0
    if integRange[1] > -999.8: upper = np.exp(-integRange[1]/tau)
    return lower-upper
def expoNorm(t, tau, integRange):
    return expo(t, tau)/expoIntegral(t, tau, integRange)

We could continue with these  functions, but it turns out to be more convenient to cast them into a class. The reason is that it allows us to write routines like draw(), below, that will plot any function, no matter how many parameters, as long as the class has the __call__ and the mini, maxi routines defined.

In [ ]:
class expoClass:
    def __init__(self, tau=1.5, funcRange=(0.0, 100)):
        self._tau=tau
        self._range=funcRange
    def setTau(self,tau):
        self._tau=tau
    def interval(self):
        return self._range
    def mini(self):
        return self._range[0]
    def maxi(self):
        return self._range[1]
    def range(self):
        return self._range
    def __call__(self, t):
        #return expo(t, self._tau)
        return expoNorm(t, self._tau, self._range)

...let's create an instance of our expoClass:

In [ ]:
timeFct = expoClass(0.9, (0, 4))
# timeFct is now defined such that timeFct(t) will call whetever is in
# the __call__ routine of expoClass, currently return expoNorm(t, self._tau, self._range)

... and draw it:

In [ ]:
def draw(somethingLikeExpoClass, scale=1, numPoints=1000):
    mini=somethingLikeExpoClass.mini()
    maxi=somethingLikeExpoClass.maxi()
    tArray=np.arange(mini, maxi, (maxi-mini)/numPoints)
    plt.plot(tArray,scale*somethingLikeExpoClass(tArray))

draw(timeFct)

Now we define a function that calculates the chi2 between the histogram and the theoretical distribution

**How do I compute the errors for each bin?**

**How do I handle the normalization?**


In [ ]:
def chi2(h, functionClass):
    ## Note that the function is normalised to one. To get the expected number of events, we
    ## will need the total number of events, and the binwidth.
    N=fsum(h[0])
    chi2sum=0
    for Nbin, bin_low, bin_up in zip(h[0], h[1], h[1][1:]):
        ##print(Nbins, ", ", bin_low, ", ", bin_up)
        binCentre=0.5*(bin_low + bin_up)
        binWidth= bin_up - bin_low
        theory=N*binWidth*functionClass(binCentre)
        measured=Nbin
        error_squared=theory
        chi2sum += (theory-measured)**2/error_squared  #poissonian errors
    return chi2sum


Now we have all the ingredients to calculate the chi2. Let's do it:

In [ ]:
chi2(timeHisto,timeFct)

Let's try different values of tau

In [ ]:
timeFct.setTau(1)
print(chi2(timeHisto, timeFct))
timeFct.setTau(1.5)
print(chi2(timeHisto, timeFct))
timeFct.setTau(0.9)
print(chi2(timeHisto, timeFct))

That was informative, but we need to make this a bit more systematic:

In [ ]:
class Chi2Scan:
    def __init__(self, h, fcn):
        self._h = h
        self._fcn =fcn
        self._tArray=[]
        self._chi2Array=[]
        self._minChi2=-999.9
        self._taumin=-999.9
        self._error_low=-999.9
        self._error_high=-999.9
    def Chi2_of_tau(self, tau):
        self._fcn.setTau(tau)
        return chi2(self._h, self._fcn)

    def scan(self, interval, numsteps):
        stepsize = (interval[1]-interval[0])/numsteps
        self._tArray=np.arange(interval[0], interval[1], stepsize)
        self._chi2Array=self.Chi2_of_tau(self._tArray)

    def plot(self, interval=(0,0), numsteps=0):
        if numsteps > 0: # only then re-do scan
            self.scan(interval, numsteps)
        plt.plot(self._tArray,self._chi2Array)
        plt.show

    def findMin(self, interval=(0,0), numsteps=0):
        if numsteps > 0: # only then re-do scan
            self.scan(interval, numsteps)
        self._minChi2=self._chi2Array[0]
        self._taumin=self._tArray[0]
        for time, chi2 in zip(self._tArray, self._chi2Array):
            if chi2 < self._minChi2:
                self._minChi2=chi2
                self._taumin=time
        return self._taumin, self._minChi2

    def findErrors(self, interval=(0,0), numsteps=0):
        if numsteps > 0: # only then re-do scan
            self.scan(interval, numsteps)
            findMin()
        shiftedChi2 = self._chi2Array - self._minChi2 - 1
        prevChi2=shiftedChi2[0]
        prevTime=self._tArray[0]
        for time, chi2 in zip(self._tArray, shiftedChi2):
            if prevChi2*chi2 < 0:
                if time < self._taumin:
                    self._error_low = -(0.5*(time + prevTime)-self._taumin)
                if time > self._taumin:
                    self._error_high = 0.5*(time + prevTime)-self._taumin
            prevChi2=chi2
            prevTime=time

        return self._error_low, self._error_high

TimeChi2Scan=Chi2Scan(timeHisto, timeFct)
TimeChi2Scan.scan((1.3,1.7), 100)
taumin, chi2min = TimeChi2Scan.findMin()
err_lo, err_hi = TimeChi2Scan.findErrors()
TimeChi2Scan.plot()

print("min at ", taumin, " + ", err_hi, " - ", err_lo, "\nwith chi2 at min of ", chi2min)



Let's write a function that can draw the data with the fit superimposed to see if it makes any sense:

In [ ]:
def drawHistoWithFit(histo, fit):
    fig, ax = plt.subplots()
    #plt.plot(timeHisto[1][0:-1], timeHisto[0][0:], drawstyle="steps-pre")#, align='edge')
    error=np.sqrt(timeHisto[0])
    #plt.bar([0,1,2, 9], [9, 10, 11, 90], align='edge')
    ax.errorbar(timeHisto[1][0:-1], timeHisto[0][0:],
       yerr=error,fmt="r+")
    N=fsum(timeHisto[0])
    binwidth=timeHisto[1][1]-timeHisto[1][0]
    draw(timeFct, scale=N*binwidth)
    plt.show()

and use it - down't forget to set tau to the fit result:

In [ ]:
timeFct.setTau(taumin)
drawHistoWithFit(timeHisto, timeFct)
print(taumin)

How do I check if the fit is healthy?

**pull study**

In [ ]:
def pullStudy(func=timeFct, numEvents=1000, numExperiments=100
              , trueTau=1.5, scanrange=(0,0), numTimeBins=50):
    pull=[]
    pullSum=0.0
    pullSumSq=0.0
    for counter in range(numExperiments):
        simulatedData = generateExpo(trueTau, numEvents)
        histo=plt.hist(simulatedData, numTimeBins, timeFct.range()) # make histogram
        scanner=Chi2Scan(histo, func)

        localScanRange=scanrange
        if(scanrange[0] >= scanrange[1]): # if scanrage not set, make one up
            scanmin = trueTau - 8*trueTau/sqrt(numEvents)
            scanmax = trueTau + 8*trueTau/sqrt(numEvents)
            if trueTau - scanmin < 1: scanmin = trueTau-1
            if scanmax - trueTau < 1: scanmax = trueTau+1
            localScanRange=(scanmin, scanmax)

        scanner.scan(localScanRange, 1000)
        taumin, chi2min = scanner.findMin()
        #print("taumin ", taumin)
        err_lo, err_hi = scanner.findErrors()
    # note: I'll take the average error, but taking upper and lower into accoun
    # seperately, depending on whether your fit is above or below the true value
    # is more accurate - something you might want to implement.
        err = 0.5*(err_lo + err_hi)
        #print("error ", err)
        thisPull = (taumin-trueTau)/err
        pull.append(thisPull)
        pullSum += thisPull
        pullSumSq += thisPull**2

    mean=pullSum/numExperiments
    var=pullSumSq/numExperiments - mean*mean
    sigma=np.sqrt(var)
    sigma_on_mean=sigma/sqrt(numExperiments)
    sigma_on_sigma=sigma/sqrt(2*numExperiments)
    print("mean pull (should be 0)", mean, " +- ", sigma_on_mean)
    print("sigma pull (should be 1)", sigma, " +- ", sigma_on_sigma)
    plt.clf()
    plt.hist(pull, 20, (-5, 5))
    plt.show()

In [ ]:
pullStudy(timeFct, numEvents=10000, numExperiments=1000
          , trueTau=1.5, scanrange=(0.5, 2.5), numTimeBins=50)